In [1]:
import sys
import os.path
import os
import math
import datetime, time
import numpy as np
from sklearn import preprocessing, model_selection
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from scipy.misc import imread
from matplotlib.pyplot import imread
import cv2
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from PIL import Image

import lfw
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

# Data

In [2]:
DATA_PATH='D:/datasets/lfw_ytf/'
DATASET_PATH=DATA_PATH+'lfw_mtcnn_aligned'
#DATASET_PATH=DATA_PATH+'lfw_mtcnn'

In [3]:
img_extensions=['.jpg','.jpeg','.png']
def is_image(path):
    _, file_extension = os.path.splitext(path)
    return file_extension.lower() in img_extensions

def get_files(db_dir):
    return [[d,os.path.join(d,f)] for d in next(os.walk(db_dir))[1] for f in sorted(next(os.walk(os.path.join(db_dir,d)))[2]) if not f.startswith(".") and is_image(f)]

In [4]:
def get_single_image_per_class_cv(y, n_splits=10,random_state=0):
    res_cv=[]
    inds = np.arange(len(y))
    np.random.seed(random_state)
    for _ in range(n_splits):
        inds_train, inds_test = [], []

        for lbl in np.unique(y):
            tmp_inds = inds[y == lbl]
            np.random.shuffle(tmp_inds)
            last_ind=1
            #last_ind=math.ceil(len(tmp_inds)/2)
            if last_ind==0 and len(tmp_inds)>0:
                last_ind=1
            inds_train.extend(tmp_inds[:last_ind])
            inds_test.extend(tmp_inds[last_ind:])
            
        inds_train = np.array(inds_train)
        inds_test = np.array(inds_test)
    
        res_cv.append((inds_train, inds_test))
    return res_cv

def classifier_tester(classifier,x,y):
    sss=get_single_image_per_class_cv(y)
    #sss=model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
    scores=model_selection.cross_validate(classifier,x, y, scoring='accuracy',cv=sss)
    acc=scores['test_score']
    print('accuracies=',acc*100)
    print('total acc=',round(acc.mean()*100,2),round(acc.std()*100,2))
    print('test time=',scores['score_time'])

## Recognition 

In [5]:
if False:
    dirs_and_files=np.array(get_files(DATASET_PATH))
else: #LFW and YTF concatenation
    subjects = (line.rstrip('\n') for line in open('lfw_ytf_classes.txt'))
    dirs_and_files=np.array([[d,os.path.join(d,f)] for d in subjects for f in sorted(next(os.walk(os.path.join(DATASET_PATH,d)))[2]) if is_image(f)])

dirs=dirs_and_files[:,0]
files=dirs_and_files[:,1]

label_enc=preprocessing.LabelEncoder()
label_enc.fit(dirs)
y=label_enc.transform(dirs)

y_l=list(y)
indices=[i for i,el in enumerate(y_l) if y_l.count(el) > 1]
y=y[indices]
label_enc=preprocessing.LabelEncoder()
label_enc.fit(y)
y=label_enc.transform(y)
print('after loading: num_classes=',len(label_enc.classes_))

after loading: num_classes= 596


In [6]:
pca_components=128 #256
classifiers=[]
#classifiers.append(['k-NN+PCA',Pipeline(steps=[('pca', PCA(n_components=pca_components)), ('classifier', KNeighborsClassifier(n_neighbors=1,p=2))])])
classifiers.append(['k-NN',KNeighborsClassifier(n_neighbors=1,p=2)])
#classifiers.append(['linear svm',LinearSVC()])

def test_faceID():
    for cls_name,classifier in classifiers:
        print(cls_name)
        classifier_tester(classifier,X_norm,y)
        print()

## Verification

In [7]:
pairs = lfw.read_pairs(DATA_PATH+'pairs.txt')
#print(pairs.shape,pairs[:5],pairs[-5:])
# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(DATASET_PATH, pairs)
#print(len(paths), len(actual_issame), paths[:5], actual_issame[:5], paths[-5:], actual_issame[-5:])

embedding_indices=np.zeros(len(paths), dtype=int)
filepaths={}
for i,p in enumerate(paths):
    if p not in filepaths:
        filepaths[p]=len(filepaths)
    embedding_indices[i]=filepaths[p]
#print(len(filepaths),len(embedding_indices), len(actual_issame), embedding_indices[:5], actual_issame[:5], embedding_indices[-5:], actual_issame[-5:])

files4verification=[None]*len(filepaths)
for f,i in filepaths.items():
    files4verification[i]=f


D:\src_code\mobile\mobile-face-recognition\src\lfw.py:193: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pairs)


In [8]:
def print_verification_results(X):
    nrof_embeddings = len(actual_issame)*2  # nrof_pairs * nrof_images_per_pair
    embedding_size=X.shape[1]
    embeddings = np.zeros((nrof_embeddings, embedding_size))
    for i,emb_ind in enumerate(embedding_indices):
        embeddings[i,:]=X[emb_ind]

    tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=0)

    print('Accuracy: %2.5f+-%2.5f' % (np.mean(accuracy), np.std(accuracy)))
    print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))

    auc = metrics.auc(fpr, tpr)
    print('Area Under Curve (AUC): %1.5f' % auc)
    eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
    print('Equal Error Rate (EER): %1.3f' % eer)

# PyTorch

In [9]:
import torch
import torch.nn as nn
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(device)

cuda


## Our models 

In [10]:
import json
sys.path.append("../once-for-all")
from ofa.imagenet_classification.networks.mobilenet_v3 import MobileNetV3
from ofa.imagenet_classification.elastic_nn.networks import OFAMobileNetV3

def get_model_artifacts(net_folder,model_name):
    config_filepath = os.path.join(net_folder, model_name,f"{model_name}.config")
    test_pt_filepath = os.path.join(net_folder, model_name,f"{model_name}.pth")
    model_config = json.load(open(config_filepath, 'r'))
    test_checkpoint = torch.load(test_pt_filepath, map_location=torch.device('cpu'))
    filter_end_fn = lambda x : not x.endswith('total_ops') and not x.endswith('total_params')
    filter_start_fn = lambda x : not x.startswith('total_ops') and not x.startswith('total_params')
    filtered_state_dict = {key:value for key,value in test_checkpoint['state_dict'].items() if filter_start_fn(key) and filter_end_fn(key)}
    model=MobileNetV3.build_from_config(model_config)
    model.load_state_dict(filtered_state_dict)
    #torch.save(model,model_name+'.pt')
    return model

In [14]:
def get_my_model(model_name):
    print(model_name)
    if model_name=='SuperNet':
        cnn_model = OFAMobileNetV3(n_classes=9131,dropout_rate=0.,width_mult=1.2, ks_list= [3, 5, 7],expand_ratio_list=[3, 4, 6],depth_list=[2, 3, 4])
        state_dict = torch.load('../models/face_recognition/ofa_mbv3_model_best_new.pth.tar', map_location="cpu")["state_dict"]
        cnn_model.load_state_dict(state_dict)
        #print(cnn_model)
    else:
        cnn_model=get_model_artifacts('../models/face_recognition/ofa_subnets',model_name)
    cnn_model.classifier=torch.nn.Identity()
    cnn_model=cnn_model.to(device)
    cnn_model.eval()
    return cnn_model

model_names=[
    'SuperNet',

    'subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_nac_gbdt',
    'subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_nac_gbdt',
    'subnet_device_765_acc_98.3_lut_34.6ms_w12_d234_nac_gbdt',
    'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_nac_gbdt',
    'subnet_device_raspberry_acc_98.4_lut_100.0ms_w12_d234_nac_gbdt',
    'subnet_device_raspberry_acc_97.8_lut_64.0ms_w12_d234_nac_gbdt',

    #'subnet_device_865_acc_97.9_lut_12.1ms_w12_d234_reg_predictor',
    #'subnet_device_865_acc_97.7_lut_9.15ms_w12_d234_reg_predictor',
    #'subnet_device_765_acc_97.9_lut_34.6ms_w12_d234_reg_predictor',
    #'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_reg_predictor',

    #'subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_reg_gbdt',
    #'subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_reg_gbdt',
    #'subnet_device_765_acc_98.4_lut_34.6ms_w12_d234_reg_gbdt',
    #'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_reg_gbdt',

]

---------------

In [12]:
def extract_torch_features(model, files,crop_center=False, standard_preprocessing=True, snr_db=None):
    snr=None if snr_db is None else 10**(snr_db/10)
    print('SNR:',snr_db,snr)
    if standard_preprocessing:
        from torchvision import transforms
        test_transforms = transforms.Compose(
            [
                transforms.Resize((224,224)),
                #transforms.Resize((260,260)),
                #transforms.Resize((160,160)),
                #transforms.Resize((196,196)),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
            ]
        )
    else:
        mean_bgr = np.array([91.4953, 103.8827, 131.0912])
        def test_transforms(img):
            img = np.array(img.resize((224,224),Image.BILINEAR))
            img = img[:, :, ::-1]  # RGB -> BGR
            img = img.astype(np.float32)
            img -= mean_bgr
            img = img.transpose(2, 0, 1)  # C x H x W
            img = torch.from_numpy(img).float()
            return img
    X=[]
    imgs=[]
    for filepath in files:
        img = Image.open(os.path.join(DATASET_PATH,filepath))
        if False:
            img = Image.open(os.path.join(DATASET_PATH,filepath))
        else:
            #img = cv2.imread(os.path.join(DATASET_PATH,filepath))
            #img=img[:, :, ::-1] #bgr2rgb
            img=np.array(img)
            if snr is not None:
                row,col,ch= img.shape
                noise_std=math.sqrt(np.var(img)*snr)
                #gauss = np.random.normal(0,noise_std*256,(row,col,ch))
                gauss = np.random.normal(0,noise_std,(row,col,ch))
                gauss = gauss.reshape(row,col,ch)
                #print(img.shape,gauss.shape,img.dtype,gauss.dtype)
                img = img + gauss
                cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX, dtype=-1)
                #img = img.astype(np.uint8)
                #print(img.shape,img.dtype)

        #print(img.shape)
        img=Image.fromarray(img.astype(np.uint8))
        
        if crop_center:
            orig_w,orig_h=250,250
            img = img.resize((orig_w,orig_h),Image.BILINEAR)
            w1,h1=224,224#128,128
            dw=(orig_w-w1)/2
            dh=(orig_h-h1)/2
            box = (dw, dh, orig_w-dw, orig_h-dh)
            img = img.crop(box)
        #img = img.resize((224,224),Image.BILINEAR)
            
        if img.size:
            img_tensor = test_transforms(img)
            #if noise_std>0:
            #    img_tensor+=noise_std*torch.randn(*img_tensor.shape)
            imgs.append(img_tensor)
            if len(imgs)>=8: #1:#8:
                scores = model(torch.stack(imgs, dim=0).to(device))
                scores=scores.data.cpu().numpy()
                #print(scores.shape)
                if len(X)==0:
                    X=scores
                else:
                    X=np.concatenate((X,scores),axis=0)
                
                imgs=[]

    if len(imgs)>0:        
        scores = model(torch.stack(imgs, dim=0).to(device))
        scores=scores.data.cpu().numpy()
        #print(scores.shape)
        if len(X)==0:
            X=scores
        else:
            X=np.concatenate((X,scores),axis=0)
    
    return X

In [15]:
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    
    start_time = time.time()
    X=extract_torch_features(cnn_model,files)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)
    
    print('Recognition:')
    test_faceID()

SuperNet
SNR: None None
--- 166.3309633731842 seconds ---
after loading: X shape: (3739, 1536)  X_norm shape: (3739, 1536)
Recognition:
k-NN
accuracies= [99.42729876 99.42729876 96.02290805 99.17276487 99.36366529 99.14094814
 99.14094814 99.36366529 99.07731467 99.58638244]
total acc= 98.97 1.0
test time= [0.25931096 0.16093493 0.15459228 0.15309668 0.15248847 0.15458322
 0.15259171 0.15358901 0.15592527 0.1536231 ]

subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_nac_gbdt
SNR: None None
--- 15.388870239257812 seconds ---
after loading: X shape: (3739, 1536)  X_norm shape: (3739, 1536)
Recognition:
k-NN
accuracies= [99.23639835 99.33184855 90.51861279 98.91823099 98.98186446 98.44097996
 99.0136812  99.0136812  98.72733058 99.07731467]
total acc= 98.13 2.55
test time= [0.1725378  0.15632844 0.1545465  0.16260242 0.15454531 0.15961385
 0.15458632 0.15458655 0.15569091 0.15582561]

subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_nac_gbdt
SNR: None None
--- 13.135858297348022 seconds ---
a

In [22]:
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    for noise_std in np.linspace(0,1,11):
        torch.manual_seed(0)
        start_time = time.time()
        X=extract_torch_features(cnn_model,files,noise_std=noise_std)
        print('--- %s seconds ---' % (time.time() - start_time))

        X_norm=preprocessing.normalize(X,norm='l2')
        X_norm=X_norm[indices,:]
        #print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

        print('Recognition. noise_std:', noise_std)
        test_faceID()

SuperNet
--- 28.8558247089386 seconds ---
Recognition. noise_std: 0.0
k-NN
accuracies= [99.42729876 99.42729876 96.02290805 99.17276487 99.36366529 99.14094814
 99.14094814 99.36366529 99.07731467 99.58638244]
total acc= 98.97 1.0
test time= [0.17955518 0.15758061 0.15358996 0.16156673 0.1525929  0.15956879
 0.16655135 0.15459108 0.16256571 0.15957069]

--- 33.99504017829895 seconds ---
Recognition. noise_std: 0.1
k-NN
accuracies= [99.36366529 99.33184855 95.45020681 98.95004773 99.23639835 99.14094814
 99.26821508 99.20458161 99.17276487 99.30003182]
total acc= 98.84 1.14
test time= [0.16160345 0.17150807 0.15761685 0.15554857 0.17050457 0.15562606
 0.16157055 0.16455293 0.16455507 0.16755605]

--- 34.12093639373779 seconds ---
Recognition. noise_std: 0.2
k-NN
accuracies= [99.1091314  99.26821508 93.9230035  98.69551384 98.98186446 98.91823099
 99.23639835 99.04549793 98.69551384 99.17276487]
total acc= 98.5 1.54
test time= [0.19447994 0.20046377 0.20441508 0.18048215 0.17453551 0.165

--- 16.421038389205933 seconds ---
Recognition. noise_std: 0.1
k-NN
accuracies= [97.16831053 97.6773783  84.60069997 96.81832644 97.48647789 97.26376074
 97.80464524 97.51829462 97.55011136 97.58192809]
total acc= 96.15 3.86
test time= [0.19451594 0.23632765 0.24006152 0.19804072 0.25183821 0.24634027
 0.22639489 0.19006848 0.18658042 0.19643664]

--- 18.22244644165039 seconds ---
Recognition. noise_std: 0.2
k-NN
accuracies= [95.99109131 96.7865097  82.50079542 95.70474069 96.27744193 96.15017499
 96.59560929 96.37289214 96.34107541 96.72287623]
total acc= 94.94 4.16
test time= [0.19331956 0.16958427 0.17313027 0.20742679 0.17806721 0.19950318
 0.18450904 0.18849611 0.18600798 0.17653227]

--- 18.637736558914185 seconds ---
Recognition. noise_std: 0.30000000000000004
k-NN
accuracies= [93.50938594 94.97295577 79.8918231  94.24117086 94.17753738 94.05027044
 94.52752148 94.75023863 94.24117086 94.97295577]
total acc= 92.93 4.37
test time= [0.22638655 0.17222357 0.17011619 0.16722393 0.18

--- 18.11754536628723 seconds ---
Recognition. noise_std: 0.2
k-NN
accuracies= [95.45020681 96.43652561 81.48265988 95.64110722 95.76837416 95.95927458
 96.46834235 95.99109131 96.34107541 96.40470888]
total acc= 94.59 4.38
test time= [0.17453313 0.16655421 0.19052339 0.17552948 0.16858983 0.16456199
 0.1715436  0.16455221 0.16160774 0.1695075 ]

--- 16.658447265625 seconds ---
Recognition. noise_std: 0.30000000000000004
k-NN
accuracies= [93.22303532 94.8138721  79.4463888  93.76391982 93.89118676 93.85937003
 94.01845371 94.55933821 94.05027044 94.75023863]
total acc= 92.64 4.42
test time= [0.18055224 0.17153859 0.16755581 0.16356063 0.16455865 0.16056681
 0.16260362 0.16056991 0.15462923 0.16953969]

--- 16.372178554534912 seconds ---
Recognition. noise_std: 0.4
k-NN
accuracies= [88.70505886 90.42316258 75.75564747 89.85046134 90.07317849 90.29589564
 89.7231944  90.96404709 89.97772829 91.0594973 ]
total acc= 88.68 4.35
test time= [0.17157769 0.16551948 0.16855073 0.16256785 0.16551

------------

In [16]:
print('Verification')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files4verification)
    print('--- %s seconds ---' % (time.time() - start_time))

    print(X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)
    print()

Verification
SuperNet
SNR: None None
--- 326.17662835121155 seconds ---
(7701, 1536)
Accuracy: 0.99317+-0.00337
Validation rate: 0.98000+-0.01366 @ FAR=0.00133
Area Under Curve (AUC): 0.99978
Equal Error Rate (EER): 0.006

subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_nac_gbdt
SNR: None None
--- 41.714231967926025 seconds ---
(7701, 1536)
Accuracy: 0.99217+-0.00350
Validation rate: 0.98133+-0.01194 @ FAR=0.00100
Area Under Curve (AUC): 0.99970
Equal Error Rate (EER): 0.008

subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_nac_gbdt
SNR: None None
--- 32.185893297195435 seconds ---
(7701, 1536)
Accuracy: 0.99200+-0.00414
Validation rate: 0.97700+-0.01286 @ FAR=0.00100
Area Under Curve (AUC): 0.99955
Equal Error Rate (EER): 0.010

subnet_device_765_acc_98.3_lut_34.6ms_w12_d234_nac_gbdt
SNR: None None
--- 36.96312975883484 seconds ---
(7701, 1536)
Accuracy: 0.99300+-0.00400
Validation rate: 0.98133+-0.01352 @ FAR=0.00133
Area Under Curve (AUC): 0.99974
Equal Error Rate (EER): 0.007

subnet_

------------

In [114]:
print('Recognition')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    #for noise_std in np.linspace(0,0.2,5):
    for snr_db in [None,-30,-20,-10,0]:
        torch.manual_seed(0)
        np.random.seed(0)
        start_time = time.time()
        X=extract_torch_features(cnn_model,files,snr_db=snr_db)#noise_std=noise_std)
        print('--- %s seconds ---' % (time.time() - start_time))

        X_norm=preprocessing.normalize(X,norm='l2')
        X_norm=X_norm[indices,:]
        #print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

        #print('Recognition. noise_std:', noise_std)
        test_faceID()

Recognition
SuperNet
SNR: None None
--- 54.95504808425903 seconds ---
k-NN
accuracies= [99.42729876 99.42729876 96.02290805 99.17276487 99.36366529 99.14094814
 99.14094814 99.36366529 99.07731467 99.58638244]
total acc= 98.97 1.0
test time= [0.34308195 0.18550539 0.19251919 0.1535871  0.172575   0.16555643
 0.17054605 0.15957451 0.1685493  0.16555595]

SNR: -30 0.001
--- 67.66130948066711 seconds ---
k-NN
accuracies= [99.45911549 99.49093223 96.08654152 99.20458161 99.39548202 99.17276487
 99.23639835 99.26821508 99.1091314  99.42729876]
total acc= 98.99 0.97
test time= [0.16655493 0.17353702 0.16252995 0.15458679 0.15961099 0.15757751
 0.15960717 0.17453408 0.15458822 0.15454793]

SNR: -20 0.01
--- 69.7644145488739 seconds ---
k-NN
accuracies= [99.36366529 99.49093223 96.2456252  98.95004773 99.26821508 99.1091314
 99.17276487 99.14094814 99.04549793 99.39548202]
total acc= 98.92 0.9
test time= [0.17457056 0.15255046 0.16260171 0.17154169 0.16751671 0.16858912
 0.19344473 0.16954637 

In [126]:
print('Verification:')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    for snr_db in [-30,-20,-10,0,10]:
        np.random.seed(0)
        start_time = time.time()
        X=extract_torch_features(cnn_model,files4verification,snr_db=snr_db)
        print('--- %s seconds ---' % (time.time() - start_time))

        print(X.shape)
        X=preprocessing.normalize(X,norm='l2')

        print_verification_results(X)

Verification:
SuperNet
SNR: -30 0.001
--- 139.97925233840942 seconds ---
(7701, 1536)
Accuracy: 0.99317+-0.00404
Validation rate: 0.98233+-0.01136 @ FAR=0.00100
Area Under Curve (AUC): 0.99976
Equal Error Rate (EER): 0.006
SNR: -20 0.01
--- 148.48681378364563 seconds ---
(7701, 1536)
Accuracy: 0.99267+-0.00416
Validation rate: 0.98567+-0.00989 @ FAR=0.00100
Area Under Curve (AUC): 0.99972
Equal Error Rate (EER): 0.006
SNR: -10 0.1
--- 168.8863127231598 seconds ---
(7701, 1536)
Accuracy: 0.99233+-0.00343
Validation rate: 0.97367+-0.01545 @ FAR=0.00100
Area Under Curve (AUC): 0.99959
Equal Error Rate (EER): 0.007
SNR: 0 1.0
--- 171.73764657974243 seconds ---
(7701, 1536)
Accuracy: 0.96200+-0.00888
Validation rate: 0.81233+-0.03030 @ FAR=0.00100
Area Under Curve (AUC): 0.99311
Equal Error Rate (EER): 0.039
SNR: 10 10.0
--- 157.8727090358734 seconds ---
(7701, 1536)
Accuracy: 0.54867+-0.02289
Validation rate: 0.00200+-0.00400 @ FAR=0.00100
Area Under Curve (AUC): 0.57371
Equal Error Rate (

In [132]:
print('Verification:')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    for snr_db in [-30,-20,-10,0,10]:
        np.random.seed(0)
        start_time = time.time()
        X=extract_torch_features(cnn_model,files4verification,snr_db=snr_db)
        print('--- %s seconds ---' % (time.time() - start_time))

        print(X.shape)
        X=preprocessing.normalize(X,norm='l2')

        print_verification_results(X)

Verification:
SuperNet
SNR: -30 0.001
--- 312.69103169441223 seconds ---
(7701, 1536)
Accuracy: 0.99333+-0.00401
Validation rate: 0.98433+-0.01075 @ FAR=0.00100
Area Under Curve (AUC): 0.99973
Equal Error Rate (EER): 0.006
SNR: -20 0.01
--- 136.8758683204651 seconds ---
(7701, 1536)
Accuracy: 0.99300+-0.00407
Validation rate: 0.98033+-0.01251 @ FAR=0.00100
Area Under Curve (AUC): 0.99967
Equal Error Rate (EER): 0.006
SNR: -10 0.1
--- 137.44214248657227 seconds ---
(7701, 1536)
Accuracy: 0.99400+-0.00318
Validation rate: 0.97967+-0.01100 @ FAR=0.00100
Area Under Curve (AUC): 0.99956
Equal Error Rate (EER): 0.007
SNR: 0 1.0
--- 142.1846706867218 seconds ---
(7701, 1536)
Accuracy: 0.97900+-0.00616
Validation rate: 0.91633+-0.01986 @ FAR=0.00100
Area Under Curve (AUC): 0.99802
Equal Error Rate (EER): 0.020
SNR: 10 10.0
--- 139.30638432502747 seconds ---
(7701, 1536)
Accuracy: 0.54483+-0.01874
Validation rate: 0.00533+-0.00618 @ FAR=0.00100
Area Under Curve (AUC): 0.57143
Equal Error Rate (

In [105]:
print('Verification')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    for noise_std in np.linspace(0,0.2,5):
        torch.manual_seed(0)
        start_time = time.time()
        X=extract_torch_features(cnn_model,files4verification,noise_std=noise_std)
        print('--- %s seconds ---' % (time.time() - start_time))

        print('noise_std:', noise_std, X.shape)
        X=preprocessing.normalize(X,norm='l2')

        print_verification_results(X)
        print()

Verification
SuperNet
--- 381.64523935317993 seconds ---
noise_std: 0.0 (7701, 1536)
Accuracy: 0.99317+-0.00337
Validation rate: 0.98000+-0.01366 @ FAR=0.00133
Area Under Curve (AUC): 0.99978
Equal Error Rate (EER): 0.006

--- 153.8184413909912 seconds ---
noise_std: 0.05 (7701, 1536)
Accuracy: 0.99317+-0.00383
Validation rate: 0.97967+-0.01251 @ FAR=0.00100
Area Under Curve (AUC): 0.99964
Equal Error Rate (EER): 0.007

--- 148.51195883750916 seconds ---
noise_std: 0.1 (7701, 1536)
Accuracy: 0.99050+-0.00409
Validation rate: 0.97133+-0.01477 @ FAR=0.00100
Area Under Curve (AUC): 0.99943
Equal Error Rate (EER): 0.010

--- 154.0610044002533 seconds ---
noise_std: 0.15000000000000002 (7701, 1536)
Accuracy: 0.98333+-0.00471
Validation rate: 0.92867+-0.02217 @ FAR=0.00133
Area Under Curve (AUC): 0.99885
Equal Error Rate (EER): 0.015

--- 151.63864493370056 seconds ---
noise_std: 0.2 (7701, 1536)
Accuracy: 0.97150+-0.00603
Validation rate: 0.88600+-0.02736 @ FAR=0.00133
Area Under Curve (AUC

# EfficientNets 

In [25]:
import timm
enet_models={}
if False:
    cnn_model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
    cnn_model.classifier=torch.nn.Identity()
    cnn_model.load_state_dict(torch.load('D:/src_code/DNN_models/state_vggface2_enet0_new.pt')) #_new
    enet_models['enet-b0']=cnn_model

    cnn_model = torch.load('D:/src_code/DNN_models/vggface2_enet25.pt')
    cnn_model.classifier=torch.nn.Identity()
    enet_models['enet-b2']=cnn_model
else:
    hsemotion_path='D:/src_code/my/face-emotion-recognition/models/pretrained_faces/'
    cnn_model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
    cnn_model.classifier=torch.nn.Identity()
    cnn_model.load_state_dict(torch.load(hsemotion_path+'state_vggface2_enet0_new.pt')) #_new
    enet_models['enet-b0']=cnn_model

    cnn_model=timm.create_model('tf_efficientnet_b2_ns', pretrained=False)
    cnn_model.classifier=torch.nn.Identity()
    cnn_model.load_state_dict(torch.load(hsemotion_path+'state_vggface2_enet2.pt')) #_new
    enet_models['enet-b2']=cnn_model

In [26]:
print('Verification:')
for dataset in ['lfw_mtcnn_aligned', 'lfw_mtcnn']:
    print(dataset)
    DATASET_PATH=DATA_PATH+dataset
    for model_name in enet_models:
        print(model_name)
        cnn_model=enet_models[model_name]
        cnn_model=cnn_model.to(device)
        cnn_model.eval()
        for snr_db in [-30,-20,-10,0,10]:
            np.random.seed(0)
            start_time = time.time()
            X=extract_torch_features(cnn_model,files4verification,snr_db=snr_db)
            print('--- %s seconds ---' % (time.time() - start_time))

            print(X.shape)
            X=preprocessing.normalize(X,norm='l2')

            print_verification_results(X)
        print()
    print()

Verification:
lfw_mtcnn_aligned
enet-b0
SNR: -30 0.001
--- 121.47695446014404 seconds ---
(7701, 1280)
Accuracy: 0.97367+-0.00869
Validation rate: 0.86200+-0.03715 @ FAR=0.00133
Area Under Curve (AUC): 0.99629
Equal Error Rate (EER): 0.027
SNR: -20 0.01
--- 112.1001889705658 seconds ---
(7701, 1280)
Accuracy: 0.96983+-0.00689
Validation rate: 0.85933+-0.04396 @ FAR=0.00133
Area Under Curve (AUC): 0.99611
Equal Error Rate (EER): 0.029
SNR: -10 0.1
--- 110.83856129646301 seconds ---
(7701, 1280)
Accuracy: 0.96283+-0.00679
Validation rate: 0.76033+-0.02877 @ FAR=0.00100
Area Under Curve (AUC): 0.99254
Equal Error Rate (EER): 0.038
SNR: 0 1.0
--- 110.25811219215393 seconds ---
(7701, 1280)
Accuracy: 0.78833+-0.01765
Validation rate: 0.15467+-0.02829 @ FAR=0.00100
Area Under Curve (AUC): 0.82847
Equal Error Rate (EER): 0.231
SNR: 10 10.0
--- 108.52767992019653 seconds ---
(7701, 1280)
Accuracy: 0.54583+-0.01739
Validation rate: 0.00367+-0.00407 @ FAR=0.00100
Area Under Curve (AUC): 0.57188


## SeNet (VggFace2) 

In [128]:
cnn_model = torch.load('D:/src_code/DNN_models/vggface_senet50_1.pt')
cnn_model=cnn_model.to(device)
cnn_model.eval()

SENet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)

In [39]:
start_time = time.time()
X=extract_torch_features(cnn_model,files, standard_preprocessing=False)
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\2271549186.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 42.60752272605896 seconds ---
after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)


In [41]:
print('Recognition:')
test_faceID()

Recognition:
k-NN+PCA
accuracies= [98.95004773 98.79096405 84.66433344 98.02736239 98.63188037 98.25007954
 98.79096405 98.85459752 98.21826281 98.88641425]
total acc= 97.21 4.19
test time= [0.10276651 0.10775042 0.10376048 0.10375953 0.10376334 0.10475445
 0.10176659 0.10177088 0.09973216 0.10176492]

k-NN
accuracies= [98.91823099 98.91823099 84.6325167  98.1228126  98.50461343 98.25007954
 98.82278078 98.82278078 98.37734648 98.91823099]
total acc= 97.23 4.21
test time= [0.19647241 0.19351745 0.20142484 0.20349264 0.19643807 0.20541239
 0.19843364 0.20046544 0.18350315 0.18451214]

linear svm
accuracies= [98.88641425 98.91823099 87.97327394 98.40916322 98.5682469  98.53643016
 98.88641425 98.88641425 98.69551384 99.0136812 ]
total acc= 97.68 3.24
test time= [0.0748024  0.07675815 0.07280827 0.0828011  0.07882667 0.07879925
 0.07081342 0.06882143 0.06784153 0.07679391]



In [49]:
start_time = time.time()
X=extract_torch_features(cnn_model,files4verification, standard_preprocessing=False)
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\2271549186.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 78.83677887916565 seconds ---
Verification
(7701, 2048)
Accuracy: 0.99367+-0.00371
Validation rate: 0.97833+-0.01276 @ FAR=0.00100
Area Under Curve (AUC): 0.99964
Equal Error Rate (EER): 0.006


-------------------

In [30]:
print('Recognition:')
for noise_std in np.linspace(0,1,11):
    torch.manual_seed(0)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files, standard_preprocessing=False,noise_std=noise_std*128)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('noise_std:',noise_std,'after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)
    test_faceID()

Recognition:


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 35.89466214179993 seconds ---
noise_std: 0.0 after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)
k-NN
accuracies= [98.91823099 98.91823099 84.6325167  98.1228126  98.50461343 98.25007954
 98.82278078 98.82278078 98.37734648 98.91823099]
total acc= 97.23 4.21
test time= [0.20349884 0.21542454 0.20699    0.204247   0.2353313  0.20448947
 0.20345116 0.21201658 0.20492005 0.20154309]



C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 42.63837146759033 seconds ---
noise_std: 0.1 after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)
k-NN
accuracies= [98.44097996 98.5682469  82.11899459 97.35921094 98.02736239 98.02736239
 98.47279669 98.6636971  97.77282851 98.34552975]
total acc= 96.58 4.84
test time= [0.21520495 0.20648646 0.20754933 0.21298361 0.25328422 0.20745635
 0.20545101 0.20646119 0.20532918 0.21293521]



...

--- 43.183926582336426 seconds ---
noise_std: 0.8 after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)
k-NN
accuracies= [47.94782055 53.13394846 47.31148584 47.50238626 48.55233853 46.07063315
 56.92013999 58.03372574 49.92045816 52.87941457]
total acc= 50.83 3.99
test time= [0.211869   0.22041011 0.22938752 0.21838069 0.24235177 0.20947433
 0.21837926 0.22041845 0.23339939 0.21545672]



C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 43.27844524383545 seconds ---
noise_std: 0.9 after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)
k-NN
accuracies= [37.25739739 40.02545339 37.83009863 37.38466433 36.20744512 35.22112631
 43.55711104 45.05249761 38.05281578 42.12535794]
total acc= 39.27 3.12
test time= [0.21610832 0.25706673 0.22398376 0.21549034 0.21442628 0.23537111
 0.21741152 0.20599079 0.20893741 0.21837664]



C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 43.688854694366455 seconds ---
noise_std: 1.0 after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)
k-NN
accuracies= [27.6169265  29.01686287 27.90327712 27.58510977 25.77155584 26.24880687
 30.51224944 32.19853643 27.23512568 29.14412981]
total acc= 28.32 1.85
test time= [0.2442894  0.22295427 0.27722311 0.22938824 0.23437428 0.22939801
 0.21192884 0.27935052 0.24434924 0.23537159]



In [125]:
print('Verification:')
for snr_db in [-30,-20,-10,0,10]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files4verification, standard_preprocessing=False,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    print('noise_std:',X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification:
SNR: -30 0.001


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 171.6140160560608 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.99267+-0.00300
Validation rate: 0.97767+-0.01221 @ FAR=0.00100
Area Under Curve (AUC): 0.99963
Equal Error Rate (EER): 0.006
SNR: -20 0.01


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 180.8134105205536 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.99333+-0.00357
Validation rate: 0.97833+-0.01344 @ FAR=0.00100
Area Under Curve (AUC): 0.99961
Equal Error Rate (EER): 0.006
SNR: -10 0.1


...

--- 182.16487741470337 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.99250+-0.00461
Validation rate: 0.95900+-0.01868 @ FAR=0.00133
Area Under Curve (AUC): 0.99953
Equal Error Rate (EER): 0.007
SNR: 0 1.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 181.39226913452148 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.96417+-0.00772
Validation rate: 0.78500+-0.02971 @ FAR=0.00100
Area Under Curve (AUC): 0.99455
Equal Error Rate (EER): 0.035
SNR: 10 10.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 184.01914715766907 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.54383+-0.01466
Validation rate: 0.00367+-0.00314 @ FAR=0.00133
Area Under Curve (AUC): 0.55370
Equal Error Rate (EER): 0.476


In [129]:
print('Verification:')
for snr_db in [-30,-20,-10,0,10]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files4verification, standard_preprocessing=False,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    print('noise_std:',X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification:
SNR: -30 0.001


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 482.1131582260132 seconds ---
noise_std: (7701, 2048)
Accuracy: 0.99017+-0.00404
Validation rate: 0.95467+-0.01714 @ FAR=0.00100
Area Under Curve (AUC): 0.99941
Equal Error Rate (EER): 0.010
SNR: -20 0.01


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 164.81812977790833 seconds ---
noise_std: (7701, 2048)
Accuracy: 0.98917+-0.00430
Validation rate: 0.95467+-0.01962 @ FAR=0.00100
Area Under Curve (AUC): 0.99938
Equal Error Rate (EER): 0.011
SNR: -10 0.1


...

--- 168.03555178642273 seconds ---
noise_std: (7701, 2048)
Accuracy: 0.98900+-0.00606
Validation rate: 0.89733+-0.02560 @ FAR=0.00100
Area Under Curve (AUC): 0.99908
Equal Error Rate (EER): 0.012
SNR: 0 1.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 171.034508228302 seconds ---
noise_std: (7701, 2048)
Accuracy: 0.95933+-0.00624
Validation rate: 0.75933+-0.04462 @ FAR=0.00133
Area Under Curve (AUC): 0.99356
Equal Error Rate (EER): 0.042
SNR: 10 10.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\1432965392.py:21: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 170.46858024597168 seconds ---
noise_std: (7701, 2048)
Accuracy: 0.56217+-0.01265
Validation rate: 0.01267+-0.00359 @ FAR=0.00100
Area Under Curve (AUC): 0.58626
Equal Error Rate (EER): 0.450


In [31]:
print('Verification')
for noise_std in np.linspace(0,1,11):
    torch.manual_seed(0)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files4verification, standard_preprocessing=False,noise_std=noise_std*128)
    print('--- %s seconds ---' % (time.time() - start_time))

    print('noise_std:',noise_std,X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 82.10212469100952 seconds ---
noise_std: 0.0 (7701, 2048)
Accuracy: 0.99367+-0.00371
Validation rate: 0.97833+-0.01276 @ FAR=0.00100
Area Under Curve (AUC): 0.99964
Equal Error Rate (EER): 0.006


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 88.8873062133789 seconds ---
noise_std: 0.1 (7701, 2048)
Accuracy: 0.99400+-0.00374
Validation rate: 0.97333+-0.01491 @ FAR=0.00100
Area Under Curve (AUC): 0.99958
Equal Error Rate (EER): 0.006


...

--- 89.57867407798767 seconds ---
noise_std: 0.8 (7701, 2048)
Accuracy: 0.91850+-0.01042
Validation rate: 0.42333+-0.04480 @ FAR=0.00100
Area Under Curve (AUC): 0.97663
Equal Error Rate (EER): 0.078


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 89.17948889732361 seconds ---
noise_std: 0.9 (7701, 2048)
Accuracy: 0.89733+-0.00800
Validation rate: 0.19433+-0.06004 @ FAR=0.00133
Area Under Curve (AUC): 0.95767
Equal Error Rate (EER): 0.103


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_13996\302545199.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 88.74858546257019 seconds ---
noise_std: 1.0 (7701, 2048)
Accuracy: 0.85100+-0.01436
Validation rate: 0.01367+-0.00960 @ FAR=0.00133
Area Under Curve (AUC): 0.92302
Equal Error Rate (EER): 0.151


# Insightface

In [130]:
import insightface
#model_path='C:/Users/avsavchenko/.insightface/models/buffalo_l/w600k_r50.onnx'
model_path='D:/src_code/DNN_models/age_gender/insightface/models/vgg2_r50_pfc.onnx'
cnn_model = insightface.model_zoo.get_model(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
cnn_model.prepare(ctx_id=0)
print(cnn_model)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


d:\program files\python38\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."


In [45]:
def extract_insightface_features(model,img_filepath,noise_std=0.0):
    img = cv2.imread(img_filepath)
    img=img[:, :, ::-1] #bgr2rgb
    if noise_std>0:
        row,col,ch= img.shape
        gauss = np.random.normal(0,noise_std*256,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        #print(img.shape,gauss.shape,img.dtype,gauss.dtype)
        img = img + gauss
        cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX, dtype=-1)
        img = img.astype(np.uint8)
        #print(img.shape,img.dtype)
    embeddings = model.get_feat(img)
    if embeddings is None:
        print(img_filepath)
    #print(embeddings.shape)
    return embeddings[0]

In [118]:
def extract_insightface_features_all(model,files,snr_db=None):
    snr=None if snr_db is None else 10**(snr_db/10)
    print('SNR:',snr_db,snr)
    imgs=[]
    X=[]
    for filepath in files:
        img = cv2.imread(os.path.join(DATASET_PATH,filepath))
            
        if img.size:
            img=img[:, :, ::-1] #bgr2rgb
            if snr is not None:
                row,col,ch= img.shape
                noise_std=math.sqrt(np.var(img)*snr)
                gauss = np.random.normal(0,noise_std,(row,col,ch))
                gauss = gauss.reshape(row,col,ch)
                #print(img.shape,gauss.shape,img.dtype,gauss.dtype)
                img = img + gauss
                cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX, dtype=-1)
                img = img.astype(np.uint8)
                #print(img.shape,img.dtype)
            imgs.append(img)
            if len(imgs)>=8: #1:#8:
                embeddings = model.get_feat(imgs)
                if embeddings is None:
                    print(img_filepath)
                #print(scores.shape)
                if len(X)==0:
                    X=embeddings
                else:
                    X=np.concatenate((X,embeddings),axis=0)
                
                imgs=[]

    if len(imgs)>0:        
        embeddings = model.get_feat(imgs)
        if embeddings is None:
            print(img_filepath)
        #print(scores.shape)
        if len(X)==0:
            X=embeddings
        else:
            X=np.concatenate((X,embeddings),axis=0)
    
    return X


In [52]:
def extract_insightface_features_all(model,files,noise_std=0.0):
    imgs=[]
    X=[]
    for filepath in files:
        img = cv2.imread(os.path.join(DATASET_PATH,filepath))
            
        if img.size:
            img=img[:, :, ::-1] #bgr2rgb
            if noise_std>0:
                row,col,ch= img.shape
                gauss = np.random.normal(0,noise_std*256,(row,col,ch))
                gauss = gauss.reshape(row,col,ch)
                #print(img.shape,gauss.shape,img.dtype,gauss.dtype)
                img = img + gauss
                cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX, dtype=-1)
                img = img.astype(np.uint8)
                #print(img.shape,img.dtype)
            imgs.append(img)
            if len(imgs)>=8: #1:#8:
                embeddings = model.get_feat(imgs)
                if embeddings is None:
                    print(img_filepath)
                #print(scores.shape)
                if len(X)==0:
                    X=embeddings
                else:
                    X=np.concatenate((X,embeddings),axis=0)
                
                imgs=[]

    if len(imgs)>0:        
        embeddings = model.get_feat(imgs)
        if embeddings is None:
            print(img_filepath)
        #print(scores.shape)
        if len(X)==0:
            X=embeddings
        else:
            X=np.concatenate((X,embeddings),axis=0)
    
    return X


In [55]:
start_time = time.time()
X=np.array([extract_insightface_features(cnn_model,os.path.join(DATASET_PATH,filepath)) for filepath in files])
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

print('Recognition:')
test_faceID()

--- 523.4497258663177 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
Recognition:
k-NN+PCA
accuracies= [99.17276487 98.98186446 98.63188037 99.0136812  98.95004773 99.14094814
 99.1091314  98.91823099 99.14094814 99.26821508]
total acc= 99.03 0.17
test time= [0.09482813 0.09382486 0.09677649 0.09179568 0.09578252 0.09574437
 0.0988121  0.09225655 0.0977385  0.09474683]

k-NN
accuracies= [99.30003182 99.33184855 98.88641425 99.33184855 99.20458161 99.14094814
 99.1091314  99.17276487 99.30003182 99.49093223]
total acc= 99.23 0.16
test time= [0.10568261 0.11773086 0.10571599 0.10622478 0.107301   0.10942411
 0.10870934 0.10779071 0.10775208 0.10622168]

linear svm
accuracies= [99.26821508 99.42729876 98.69551384 99.26821508 99.26821508 99.17276487
 99.07731467 99.23639835 99.30003182 99.5545657 ]
total acc= 99.23 0.22
test time= [0.02994299 0.02800465 0.02993798 0.02991915 0.02692151 0.025038
 0.03490615 0.03191566 0.02501273 0.02595019]



In [56]:
start_time = time.time()
X=np.array([extract_insightface_features(cnn_model,os.path.join(DATASET_PATH,filepath)) for filepath in files4verification])
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

--- 1063.6578254699707 seconds ---
Verification
(7701, 512)
Accuracy: 0.99217+-0.00373
Validation rate: 0.98567+-0.00978 @ FAR=0.00100
Area Under Curve (AUC): 0.99929
Equal Error Rate (EER): 0.006


------------

In [119]:
print('Recognition:')
for snr_db in [None,-30,-20,-10,0]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

    test_faceID()    

Recognition:
SNR: None None
--- 341.38093423843384 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
k-NN
accuracies= [99.30003182 99.33184855 98.88641425 99.33184855 99.20458161 99.14094814
 99.1091314  99.17276487 99.30003182 99.49093223]
total acc= 99.23 0.16
test time= [0.10667634 0.10176039 0.10073137 0.10671663 0.10472131 0.10069394
 0.11270213 0.09775734 0.10970497 0.10172629]

SNR: -30 0.001
--- 360.8581278324127 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
k-NN
accuracies= [99.33184855 99.36366529 98.91823099 99.30003182 99.17276487 99.17276487
 99.20458161 99.20458161 99.33184855 99.49093223]
total acc= 99.25 0.15
test time= [0.10272336 0.11469221 0.10268641 0.09678292 0.09573889 0.09574389
 0.09869885 0.09973145 0.09774113 0.09969425]

SNR: -20 0.01
--- 356.17740201950073 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
k-NN
accuracies= [99.36366529 99.42729876 98.88641425 99.20458161 99.0773146

In [120]:
print('Recognition:')
for snr_db in [10]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

    test_faceID()    

Recognition:
SNR: 10 10.0
--- 355.68193769454956 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
k-NN
accuracies= [5.66337894 3.14985682 3.62710786 5.21794464 3.37257397 3.53165765
 4.13617563 3.37257397 3.69074133 4.6452434 ]
total acc= 4.04 0.81
test time= [0.09677482 0.09674382 0.0957489  0.09873199 0.09673691 0.09670258
 0.0937891  0.09574437 0.09773922 0.09969544]



In [121]:
print('Verification:')
for snr_db in [-30,-20,-10,0,10]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files4verification,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    print(X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification:
SNR: -30 0.001
--- 977.0268676280975 seconds ---
(7701, 512)
Accuracy: 0.99217+-0.00373
Validation rate: 0.98600+-0.00975 @ FAR=0.00100
Area Under Curve (AUC): 0.99931
Equal Error Rate (EER): 0.006
SNR: -20 0.01
--- 783.7478063106537 seconds ---
(7701, 512)
Accuracy: 0.99383+-0.00409
Validation rate: 0.98533+-0.00957 @ FAR=0.00100
Area Under Curve (AUC): 0.99930
Equal Error Rate (EER): 0.006
SNR: -10 0.1
--- 740.9596321582794 seconds ---
(7701, 512)
Accuracy: 0.99283+-0.00435
Validation rate: 0.97800+-0.01002 @ FAR=0.00133
Area Under Curve (AUC): 0.99918
Equal Error Rate (EER): 0.008
SNR: 0 1.0
--- 729.3147025108337 seconds ---
(7701, 512)
Accuracy: 0.96950+-0.00592
Validation rate: 0.81500+-0.02918 @ FAR=0.00100
Area Under Curve (AUC): 0.99430
Equal Error Rate (EER): 0.032
SNR: 10 10.0
--- 722.7051177024841 seconds ---
(7701, 512)
Accuracy: 0.58250+-0.02032
Validation rate: 0.00900+-0.00633 @ FAR=0.00133
Area Under Curve (AUC): 0.61305
Equal Error Rate (EER): 0.422


In [131]:
print('Verification:')
for snr_db in [-30,-20,-10,0,10]:
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files4verification,snr_db=snr_db)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    print(X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification:
SNR: -30 0.001
--- 710.1077735424042 seconds ---
(7701, 512)
Accuracy: 0.96050+-0.00978
Validation rate: 0.83433+-0.02226 @ FAR=0.00100
Area Under Curve (AUC): 0.99221
Equal Error Rate (EER): 0.038
SNR: -20 0.01
--- 704.7772414684296 seconds ---
(7701, 512)
Accuracy: 0.96433+-0.00904
Validation rate: 0.83667+-0.02385 @ FAR=0.00133
Area Under Curve (AUC): 0.99202
Equal Error Rate (EER): 0.038
SNR: -10 0.1
--- 704.8402280807495 seconds ---
(7701, 512)
Accuracy: 0.95700+-0.01145
Validation rate: 0.81600+-0.02882 @ FAR=0.00100
Area Under Curve (AUC): 0.98920
Equal Error Rate (EER): 0.045
SNR: 0 1.0
--- 711.0272479057312 seconds ---
(7701, 512)
Accuracy: 0.89283+-0.01515
Validation rate: 0.49433+-0.02704 @ FAR=0.00100
Area Under Curve (AUC): 0.95581
Equal Error Rate (EER): 0.111
SNR: 10 10.0
--- 717.1032061576843 seconds ---
(7701, 512)
Accuracy: 0.56550+-0.01256
Validation rate: 0.00933+-0.00554 @ FAR=0.00133
Area Under Curve (AUC): 0.58273
Equal Error Rate (EER): 0.441


In [53]:
print('Recognition:')
for noise_std in np.linspace(0,1,11):
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files,noise_std=noise_std)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape,'noise_std:',noise_std)

    test_faceID()

Recognition:
--- 327.1502935886383 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512) noise_std: 0.0
k-NN
accuracies= [99.30003182 99.33184855 98.88641425 99.33184855 99.20458161 99.14094814
 99.1091314  99.17276487 99.30003182 99.49093223]
total acc= 99.23 0.16
test time= [0.09478164 0.09874034 0.10072541 0.09674263 0.09670186 0.09977388
 0.10069203 0.10272646 0.10276222 0.09972954]

--- 341.94722056388855 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512) noise_std: 0.1
k-NN
accuracies= [98.5682469  98.82278078 97.90009545 98.53643016 98.37734648 98.31371301
 98.34552975 98.28189628 98.40916322 98.75914731]
total acc= 98.43 0.25
test time= [0.09574819 0.09976244 0.11170268 0.09574246 0.09777951 0.10273004
 0.10176444 0.10874724 0.13966227 0.09977341]

--- 352.4792697429657 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512) noise_std: 0.2
k-NN
accuracies= [92.52306713 92.52306713 79.12822144 91.34584792 92.4594336

In [108]:
print('Verification:')
for noise_std in np.linspace(0,0.2,5):
    np.random.seed(0)
    start_time = time.time()
    X=extract_insightface_features_all(cnn_model,files4verification,noise_std=noise_std)
    print('--- %s seconds ---' % (time.time() - start_time))

    print('noise_std:',noise_std,X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)

Verification:
--- 663.013516664505 seconds ---
noise_std: 0.0 (7701, 512)
Accuracy: 0.99217+-0.00373
Validation rate: 0.98567+-0.00978 @ FAR=0.00100
Area Under Curve (AUC): 0.99929
Equal Error Rate (EER): 0.006
--- 718.3326072692871 seconds ---
noise_std: 0.05 (7701, 512)
Accuracy: 0.99233+-0.00410
Validation rate: 0.98200+-0.00733 @ FAR=0.00133
Area Under Curve (AUC): 0.99926
Equal Error Rate (EER): 0.008


KeyboardInterrupt: 

# Tensorflow models

In [16]:
import tensorflow as tf
import tensorflow.keras.backend as K 

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

2.4.1


In [17]:
def load_graph(frozen_graph_filename, prefix=''):
    with tf.io.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name=prefix)
    return graph

class TensorFlowInference:
    def __init__(self,frozen_graph_filename,input_tensor,output_tensor,learning_phase_tensor=None, convert2BGR=True, imageNetUtilsMean=True,additional_input_value=0):
        graph = load_graph(frozen_graph_filename,'')
        print([n.name for n in graph.as_graph_def().node if 'input' in n.name])
        
        graph_op_list=list(graph.get_operations())
        print([n.name for n in graph_op_list if 'keras_learning' in n.name])
        
        self.tf_sess=tf.compat.v1.Session(graph=graph)
        
        self.tf_input_image = graph.get_tensor_by_name(input_tensor)
        print('tf_input_image=',self.tf_input_image)
        self.tf_output_features = graph.get_tensor_by_name(output_tensor)
        print('tf_output_features=',self.tf_output_features)
        self.tf_learning_phase = graph.get_tensor_by_name(learning_phase_tensor) if learning_phase_tensor else None;
        print('tf_learning_phase=',self.tf_learning_phase)
        if self.tf_input_image.shape.dims is None:
            w=h=160
        else:
            _,w,h,_=self.tf_input_image.shape
        self.w,self.h=int(w),int(h)
        print ('input w,h',self.w,self.h,' output shape:',self.tf_output_features.shape,'convert2BGR:',convert2BGR, 'imageNetUtilsMean:',imageNetUtilsMean)
        #for n in graph.as_graph_def().node:
        #    print(n.name, n.op)
        #sys.exit(0)

        self.convert2BGR=convert2BGR
        self.imageNetUtilsMean=imageNetUtilsMean
        self.additional_input_value=additional_input_value

    def preprocess_image(self,img_filepath,crop_center,snr=None):
        if crop_center:
            orig_w,orig_h=250,250
            img = imread(img_filepath)#, mode='RGB')
            #img = misc.imresize(img, (orig_w,orig_h), interp='bilinear')
            img = np.array(Image.fromarray(img).resize((orig_w,orig_h),resample=Image.BILINEAR))
            w1,h1=128,128
            dw=(orig_w-w1)//2
            dh=(orig_h-h1)//2
            box = (dw, dh, orig_w-dw, orig_h-dh)
            img=img[dh:-dh,dw:-dw]
        else:
            img = imread(img_filepath)#, mode='RGB')
        
        #x = misc.imresize(img, (self.w,self.h), interp='bilinear').astype(float)
        x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))
        
        
        if snr is not None:
            row,col,ch= x.shape
            noise_std=math.sqrt(np.var(x)*snr)
            gauss = np.random.normal(0,noise_std,(row,col,ch))
            gauss = gauss.reshape(row,col,ch)
            x = x + gauss
            cv2.normalize(x, x, 0, 255, cv2.NORM_MINMAX, dtype=-1)
            x = x.astype(np.uint8)

        x=x.astype(float)

        if self.convert2BGR:
            # 'RGB'->'BGR'
            x = x[..., ::-1]
            # Zero-center by mean pixel
            if self.imageNetUtilsMean: #imagenet.utils caffe
                x[..., 0] -= 103.939
                x[..., 1] -= 116.779
                x[..., 2] -= 123.68
            else: #vggface-2
                x[..., 0] -= 91.4953
                x[..., 1] -= 103.8827
                x[..., 2] -= 131.0912
        else:
            #x=(x-127.5)/128.0
            x /= 127.5
            x -= 1.
            #x=x/128.0-1.0
        return x
        
    def extract_features(self,img_filepath,crop_center=False,snr=None):
        x=self.preprocess_image(img_filepath,crop_center,snr)
        x = np.expand_dims(x, axis=0)
        feed_dict={self.tf_input_image: x}
        if self.tf_learning_phase is not None:
            feed_dict[self.tf_learning_phase]=self.additional_input_value
        preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).reshape(-1)
        #preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).mean(axis=(0,1,2)).reshape(-1)
        return preds

    def extract_features_all(self,files,crop_center=False,snr=None):
        X,imgs=[],[]
        feed_dict={}
        if self.tf_learning_phase is not None:
            feed_dict[self.tf_learning_phase]=self.additional_input_value
        for filepath in files:
            x=self.preprocess_image(os.path.join(DATASET_PATH,filepath),crop_center,snr)
            imgs.append(x)
            if len(imgs)>32:
                #x = np.expand_dims(x, axis=0)
                feed_dict[self.tf_input_image]= np.array(imgs)
                preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict)#.reshape(-1)
                #preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).mean(axis=(0,1,2)).reshape(-1)
                if len(X)==0:
                    X=preds
                else:
                    X=np.concatenate((X,preds),axis=0)
                
                imgs=[]

        if len(imgs)>0:
            #x = np.expand_dims(x, axis=0)
            feed_dict[self.tf_input_image]= np.array(imgs)
            preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict)#.reshape(-1)
            #preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).mean(axis=(0,1,2)).reshape(-1)
            if len(X)==0:
                X=preds
            else:
                X=np.concatenate((X,preds),axis=0)

            imgs=[]
        return X

    def close_session(self):
        self.tf_sess.close()


## FaceNet 

In [18]:
cnn_model=TensorFlowInference('D:/src_code/DNN_models/facenet_inceptionresnet/20180402-114759.pb',input_tensor='input:0',output_tensor='embeddings:0',learning_phase_tensor='phase_train:0',convert2BGR=False) #embeddings, InceptionResnetV1/Repeat_2/block8_5/Relu, InceptionResnetV1/Repeat_1/block17_10/Relu

['input']
[]
tf_input_image= Tensor("input:0", dtype=float32)
tf_output_features= Tensor("embeddings:0", shape=(None, 512), dtype=float32)
tf_learning_phase= Tensor("phase_train:0", dtype=bool)
input w,h 160 160  output shape: (None, 512) convert2BGR: False imageNetUtilsMean: True


In [60]:
crop_center=False
start_time = time.time()
X=np.array([cnn_model.extract_features(os.path.join(DATASET_PATH,filepath),crop_center=crop_center) for filepath in files])    
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

print('Recognition:')
test_faceID()

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\1513849870.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR)).astype(float)


--- 120.72068953514099 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
Recognition:
k-NN+PCA
accuracies= [97.39102768 97.74101177 85.93700286 96.85014318 96.7865097  96.75469297
 97.6773783  97.42284442 96.97741012 97.6773783 ]
total acc= 96.12 3.41
test time= [0.08680582 0.08481026 0.08580971 0.08681345 0.088763   0.08581114
 0.09578013 0.08676887 0.08680677 0.08776951]

k-NN
accuracies= [97.39102768 97.74101177 85.93700286 96.85014318 96.7865097  96.75469297
 97.6773783  97.42284442 96.97741012 97.6773783 ]
total acc= 96.12 3.41
test time= [0.09674358 0.10072756 0.10169077 0.10475898 0.09973192 0.10172582
 0.10268569 0.10375977 0.09873319 0.10269022]

linear svm
accuracies= [96.59560929 96.91377665 88.32325803 95.73655743 96.53197582 95.89564111
 97.35921094 96.85014318 96.46834235 96.75469297]
total acc= 95.74 2.51
test time= [0.02594924 0.02695608 0.02593422 0.02593088 0.02489567 0.02596712
 0.02493906 0.02495575 0.02594829 0.02496123]



In [61]:
start_time = time.time()
X=np.array([cnn_model.extract_features(os.path.join(DATASET_PATH,filepath),crop_center=crop_center) for filepath in files4verification])    
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\1513849870.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR)).astype(float)


--- 235.93100428581238 seconds ---
Verification
(7701, 512)
Accuracy: 0.99283+-0.00527
Validation rate: 0.97667+-0.01430 @ FAR=0.00100
Area Under Curve (AUC): 0.99939
Equal Error Rate (EER): 0.008


In [19]:
print('Verification:')
for dataset in ['lfw_mtcnn_aligned', 'lfw_mtcnn']:
    print(dataset)
    DATASET_PATH=DATA_PATH+dataset
    for snr_db in [-30,-20,-10,0,10]:
        snr=None if snr_db is None else 10**(snr_db/10)
        print('SNR:',snr_db,snr)
        np.random.seed(0)
        start_time = time.time()
        X=cnn_model.extract_features_all(files4verification,crop_center=False,snr=snr)
        print('--- %s seconds ---' % (time.time() - start_time))

        print(X.shape)
        X=preprocessing.normalize(X,norm='l2')

        print_verification_results(X)
    print()

Verification:
lfw_mtcnn_aligned
SNR: -30 0.001


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 71.72998094558716 seconds ---
(7701, 512)
Accuracy: 0.99217+-0.00558
Validation rate: 0.97667+-0.01453 @ FAR=0.00100
Area Under Curve (AUC): 0.99942
Equal Error Rate (EER): 0.008
SNR: -20 0.01


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 60.546183824539185 seconds ---
(7701, 512)
Accuracy: 0.99033+-0.00567
Validation rate: 0.96900+-0.01300 @ FAR=0.00100
Area Under Curve (AUC): 0.99939
Equal Error Rate (EER): 0.009
SNR: -10 0.1


...

--- 62.195735931396484 seconds ---
(7701, 512)
Accuracy: 0.97967+-0.00623
Validation rate: 0.92167+-0.01701 @ FAR=0.00100
Area Under Curve (AUC): 0.99810
Equal Error Rate (EER): 0.019
SNR: 0 1.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 66.40251517295837 seconds ---
(7701, 512)
Accuracy: 0.84483+-0.01440
Validation rate: 0.04333+-0.01125 @ FAR=0.00100
Area Under Curve (AUC): 0.91284
Equal Error Rate (EER): 0.159
SNR: 10 10.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 59.67061114311218 seconds ---
(7701, 512)
Accuracy: 0.56083+-0.01761
Validation rate: 0.00500+-0.00307 @ FAR=0.00167
Area Under Curve (AUC): 0.58540
Equal Error Rate (EER): 0.435



In [20]:
cnn_model.close_session()

# HseFaceRec-tf

In [22]:
cnn_model=TensorFlowInference('D:/src_code/my/HSE_FaceRec_tf/age_gender_identity/age_gender_tf2_new-01-0.14-0.92.pb',input_tensor='input_1:0',output_tensor='global_pooling/Mean:0',convert2BGR=True, imageNetUtilsMean=True)

['input_1']
[]
tf_input_image= Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32)
tf_output_features= Tensor("global_pooling/Mean:0", shape=(None, 1024), dtype=float32)
tf_learning_phase= None
input w,h 224 224  output shape: (None, 1024) convert2BGR: True imageNetUtilsMean: True


In [23]:
print('Verification:')
for dataset in ['lfw_mtcnn_aligned', 'lfw_mtcnn']:
    print(dataset)
    DATASET_PATH=DATA_PATH+dataset
    for snr_db in [-30,-20,-10,0,10]:
        snr=None if snr_db is None else 10**(snr_db/10)
        print('SNR:',snr_db,snr)
        np.random.seed(0)
        start_time = time.time()
        X=cnn_model.extract_features_all(files4verification,crop_center=False,snr=snr)
        print('--- %s seconds ---' % (time.time() - start_time))

        print(X.shape)
        X=preprocessing.normalize(X,norm='l2')

        print_verification_results(X)
    print()

Verification:
lfw_mtcnn_aligned
SNR: -30 0.001


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 111.9321038722992 seconds ---
(7701, 1024)
Accuracy: 0.97950+-0.00730
Validation rate: 0.85233+-0.02221 @ FAR=0.00133
Area Under Curve (AUC): 0.99737
Equal Error Rate (EER): 0.021
SNR: -20 0.01


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 114.40218257904053 seconds ---
(7701, 1024)
Accuracy: 0.97733+-0.00779
Validation rate: 0.81733+-0.02294 @ FAR=0.00100
Area Under Curve (AUC): 0.99705
Equal Error Rate (EER): 0.023
SNR: -10 0.1


...

--- 85.24511241912842 seconds ---
(7701, 1024)
Accuracy: 0.96300+-0.00918
Validation rate: 0.80333+-0.02241 @ FAR=0.00100
Area Under Curve (AUC): 0.99256
Equal Error Rate (EER): 0.039
SNR: 0 1.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 104.12974286079407 seconds ---
(7701, 1024)
Accuracy: 0.89583+-0.00898
Validation rate: 0.44567+-0.03853 @ FAR=0.00100
Area Under Curve (AUC): 0.95397
Equal Error Rate (EER): 0.109
SNR: 10 10.0


C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_32876\199940247.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR))


--- 92.07689881324768 seconds ---
(7701, 1024)
Accuracy: 0.56267+-0.01884
Validation rate: 0.00767+-0.00448 @ FAR=0.00100
Area Under Curve (AUC): 0.58863
Equal Error Rate (EER): 0.438



In [24]:
cnn_model.close_session()